In [1]:
import pandas as pd
import polars as pl
import pyarrow as pa

data = {
    "colors": ["black", "white", "yellow"],
    "fruits": ["APPLE", "BANANA", "STRAWBERRY"],
    "animals": ["dog", "cat", "squirrel"],
}
df = pl.DataFrame(data)
print(df)

shape: (3, 3)
┌────────┬────────────┬──────────┐
│ colors ┆ fruits     ┆ animals  │
│ ---    ┆ ---        ┆ ---      │
│ str    ┆ str        ┆ str      │
╞════════╪════════════╪══════════╡
│ black  ┆ APPLE      ┆ dog      │
│ white  ┆ BANANA     ┆ cat      │
│ yellow ┆ STRAWBERRY ┆ squirrel │
└────────┴────────────┴──────────┘


## 1.1 starts_with() and ends_with()

In [2]:
print(
    df.select(
        pl.col("animals"),
        pl.col("animals").str.ends_with("t").name.suffix("_t$"),
    )
)

shape: (3, 2)
┌──────────┬────────────┐
│ animals  ┆ animals_t$ │
│ ---      ┆ ---        │
│ str      ┆ bool       │
╞══════════╪════════════╡
│ dog      ┆ false      │
│ cat      ┆ true       │
│ squirrel ┆ false      │
└──────────┴────────────┘


## 1.2 contains() and contains_any()

In [3]:
print(df.with_columns(pl.all().str.contains("rr").name.suffix("_c")))

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_c ┆ fruits_c ┆ animals_c │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str    ┆ str        ┆ str      ┆ bool     ┆ bool     ┆ bool      │
╞════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ false    ┆ false    ┆ false     │
│ white  ┆ BANANA     ┆ cat      ┆ false    ┆ false    ┆ false     │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ false    ┆ false    ┆ true      │
└────────┴────────────┴──────────┴──────────┴──────────┴───────────┘


In [4]:
print(df.with_columns(pl.all().str.contains("(?i)pp").name.suffix("_c")))

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_c ┆ fruits_c ┆ animals_c │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str    ┆ str        ┆ str      ┆ bool     ┆ bool     ┆ bool      │
╞════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ false    ┆ true     ┆ false     │
│ white  ┆ BANANA     ┆ cat      ┆ false    ┆ false    ┆ false     │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ false    ┆ false    ┆ false     │
└────────┴────────────┴──────────┴──────────┴──────────┴───────────┘


In [5]:
print(
    df.with_columns(
        pl.all().str.contains_any(["PP", "RR"]).name.suffix("_c")
    )
)

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_c ┆ fruits_c ┆ animals_c │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str    ┆ str        ┆ str      ┆ bool     ┆ bool     ┆ bool      │
╞════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ false    ┆ true     ┆ false     │
│ white  ┆ BANANA     ┆ cat      ┆ false    ┆ false    ┆ false     │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ false    ┆ true     ┆ false     │
└────────┴────────────┴──────────┴──────────┴──────────┴───────────┘


In [6]:
print(
    df.with_columns(
        pl.all()
        .str.contains_any(["PP", "RR"], ascii_case_insensitive=True)
        .name.suffix("_c")
    )
)

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_c ┆ fruits_c ┆ animals_c │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str    ┆ str        ┆ str      ┆ bool     ┆ bool     ┆ bool      │
╞════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ false    ┆ true     ┆ false     │
│ white  ┆ BANANA     ┆ cat      ┆ false    ┆ false    ┆ false     │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ false    ┆ true     ┆ true      │
└────────┴────────────┴──────────┴──────────┴──────────┴───────────┘


## 1.3 extract() and extract_all()

In [7]:
print(
    df.select(
        pl.col("fruits"),
        pl.col("fruits")
        .str.extract("BA(NA)", group_index=0)
        .alias("extract"),
        pl.col("fruits").str.extract_all("(NA)").alias("extract_all"),
    )
)

shape: (3, 3)
┌────────────┬─────────┬──────────────┐
│ fruits     ┆ extract ┆ extract_all  │
│ ---        ┆ ---     ┆ ---          │
│ str        ┆ str     ┆ list[str]    │
╞════════════╪═════════╪══════════════╡
│ APPLE      ┆ null    ┆ []           │
│ BANANA     ┆ BANA    ┆ ["NA", "NA"] │
│ STRAWBERRY ┆ null    ┆ []           │
└────────────┴─────────┴──────────────┘


## 1.4 replace(), replace_all(), replace_many()

In [8]:
print(
    df.select(
        pl.col("fruits"),
        pl.col("fruits").str.replace("A", "a").alias("r"),
        pl.col("fruits").str.replace_all("A", "a").alias("r_all"),
        pl.col("fruits")
        .str.replace_many(["A", "P"], ["a", "p"])
        .alias("r_many_list"),
        pl.col("fruits")
        .str.replace_many({"A": "a", "P": "p"})
        .alias("r_many_dict"),
    )
)

shape: (3, 5)
┌────────────┬────────────┬────────────┬─────────────┬─────────────┐
│ fruits     ┆ r          ┆ r_all      ┆ r_many_list ┆ r_many_dict │
│ ---        ┆ ---        ┆ ---        ┆ ---         ┆ ---         │
│ str        ┆ str        ┆ str        ┆ str         ┆ str         │
╞════════════╪════════════╪════════════╪═════════════╪═════════════╡
│ APPLE      ┆ aPPLE      ┆ aPPLE      ┆ appLE       ┆ appLE       │
│ BANANA     ┆ BaNANA     ┆ BaNaNa     ┆ BaNaNa      ┆ BaNaNa      │
│ STRAWBERRY ┆ STRaWBERRY ┆ STRaWBERRY ┆ STRaWBERRY  ┆ STRaWBERRY  │
└────────────┴────────────┴────────────┴─────────────┴─────────────┘


## 1.5 Case conversion

In [9]:
print(
    df.select(
        pl.col("colors").str.to_uppercase(),
        pl.col("fruits").str.to_lowercase(),
        pl.col("animals").str.to_titlecase(),
    )
)

shape: (3, 3)
┌────────┬────────────┬──────────┐
│ colors ┆ fruits     ┆ animals  │
│ ---    ┆ ---        ┆ ---      │
│ str    ┆ str        ┆ str      │
╞════════╪════════════╪══════════╡
│ BLACK  ┆ apple      ┆ Dog      │
│ WHITE  ┆ banana     ┆ Cat      │
│ YELLOW ┆ strawberry ┆ Squirrel │
└────────┴────────────┴──────────┘


## 1.6 String length

In [10]:
with pl.Config(tbl_rows=20):
    _ = pl.DataFrame({"col": list("2025IThome鐵人賽")}).select(
        pl.col("col"),
        pl.col("col").str.len_bytes().alias("n_bytes"),
        pl.col("col").str.len_chars().alias("n_chars"),
    )
    print(_)

shape: (13, 3)
┌─────┬─────────┬─────────┐
│ col ┆ n_bytes ┆ n_chars │
│ --- ┆ ---     ┆ ---     │
│ str ┆ u32     ┆ u32     │
╞═════╪═════════╪═════════╡
│ 2   ┆ 1       ┆ 1       │
│ 0   ┆ 1       ┆ 1       │
│ 2   ┆ 1       ┆ 1       │
│ 5   ┆ 1       ┆ 1       │
│ I   ┆ 1       ┆ 1       │
│ T   ┆ 1       ┆ 1       │
│ h   ┆ 1       ┆ 1       │
│ o   ┆ 1       ┆ 1       │
│ m   ┆ 1       ┆ 1       │
│ e   ┆ 1       ┆ 1       │
│ 鐵  ┆ 3       ┆ 1       │
│ 人  ┆ 3       ┆ 1       │
│ 賽  ┆ 3       ┆ 1       │
└─────┴─────────┴─────────┘


## 1.7 Slice

In [11]:
print(df.with_columns(pl.all().str.slice(1).name.suffix("_s")))

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬───────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_s ┆ fruits_s  ┆ animals_s │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---       ┆ ---       │
│ str    ┆ str        ┆ str      ┆ str      ┆ str       ┆ str       │
╞════════╪════════════╪══════════╪══════════╪═══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ lack     ┆ PPLE      ┆ og        │
│ white  ┆ BANANA     ┆ cat      ┆ hite     ┆ ANANA     ┆ at        │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ ellow    ┆ TRAWBERRY ┆ quirrel   │
└────────┴────────────┴──────────┴──────────┴───────────┴───────────┘


In [12]:
print(
    df.with_columns(
        pl.all()
        .str.slice(1, pl.all().str.len_chars() - 2)
        .name.suffix("_s")
    )
)

shape: (3, 6)
┌────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ colors ┆ fruits     ┆ animals  ┆ colors_s ┆ fruits_s ┆ animals_s │
│ ---    ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str    ┆ str        ┆ str      ┆ str      ┆ str      ┆ str       │
╞════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ black  ┆ APPLE      ┆ dog      ┆ lac      ┆ PPL      ┆ o         │
│ white  ┆ BANANA     ┆ cat      ┆ hit      ┆ ANAN     ┆ a         │
│ yellow ┆ STRAWBERRY ┆ squirrel ┆ ello     ┆ TRAWBERR ┆ quirre    │
└────────┴────────────┴──────────┴──────────┴──────────┴───────────┘


## codepanda

In [13]:
df_pd = pd.DataFrame({"v0_object": ["black"]}).assign(
    v1_stringarrow=lambda df_: df_.v0_object.astype(
        {"v0_object": "string[pyarrow]"}
    ),
    v2_stringpa=lambda df_: df_.v0_object.astype(
        {"v0_object": pd.ArrowDtype(pa.string())}
    ),
)

print(df_pd.dtypes)

v0_object                  object
v1_stringarrow    string[pyarrow]
v2_stringpa       string[pyarrow]
dtype: object


In [14]:
print(
    df_pd.dtypes["v1_stringarrow"] is df_pd.dtypes["v2_stringpa"],
    df_pd.dtypes["v1_stringarrow"] == df_pd.dtypes["v2_stringpa"],
)

False False
